# Week 3 Project. Segmenting and Clustering Toronto.
## Choosing the most suitable Neighbourhood.

### 1st. Importation of main libraries to the Jupiter Notebook.
We will make the imports in various cells as the speed for connection is not always available.

In [1]:
import pandas as pd
import numpy as np
import urllib.request

### 2nd. Importation of special libraries for visualization and coordinates

In [2]:
!conda install -c conda-forge geopy --yes #Needed to reinstall in order to work
from geopy.geocoders import Nominatim
import folium
import requests

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    nu

In [3]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### In part 2 of the assignment we exported the data to a CSV file in order to save steps on the 3rd part of it. So now we only have to import that data into a new dataframe.

In [4]:
dfCanadaNeigh = pd.read_csv("CanadaZIPCodeswithPosition.csv")

In [6]:
dfCanadaNeigh

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...,...
98,98,M9N,York,Weston,43.706876,-79.518188
99,99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


## Getting Connected to Foursquare to retrieve data

In [7]:
CLIENT_ID = 'P5OUT2CRIK14KOBJ3KOMSQ4C3CCAYGLSJIGB1DNNHD5JJU5P'
CLIENT_SECRET ='ZP1XXJGAVE3CYMRIJSNKZJNFJ1RI4ABDQ5WX0SD0MUVPVIP5'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P5OUT2CRIK14KOBJ3KOMSQ4C3CCAYGLSJIGB1DNNHD5JJU5P
CLIENT_SECRET:ZP1XXJGAVE3CYMRIJSNKZJNFJ1RI4ABDQ5WX0SD0MUVPVIP5


### Now as seen on previous practices we have to create a function to retrieve the data from FourSquare, that will be the venues for each neighbourhood.

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now we run the function for a Neighbourhood in Toronto (York)

In [40]:
#dfToronto=dfCanadaNeigh[dfCanadaNeigh['Borough']=='East York'].reset_index(drop=True)
#dfToronto=dfCanadaNeigh.loc[~((dfCanadaNeigh['Borough']=='East York')  & (dfCanadaNeigh['Borough']=='Scarborough'))].reset_index(drop=True)
# & (dfCanadaNeigh['Borough']=='North York') & (dfCanadaNeigh['Borough']=='Etobicoke')& (dfCanadaNeigh['Borough']=='Downtown Toronto')
dfEastYork=dfCanadaNeigh[dfCanadaNeigh['Borough']=='East York'].reset_index(drop=True)
dfNorthYork=dfCanadaNeigh[dfCanadaNeigh['Borough']=='North York'].reset_index(drop=True)
dfEtobicoke=dfCanadaNeigh[dfCanadaNeigh['Borough']=='Etobicoke'].reset_index(drop=True)
dfScarborough=dfCanadaNeigh[dfCanadaNeigh['Borough']=='Scarborough'].reset_index(drop=True)
dfDTownToronto=dfCanadaNeigh[dfCanadaNeigh['Borough']=='Downtown Toronto'].reset_index(drop=True)
dfToronto=dfDTownToronto
dfToronto=dfToronto.append(dfScarborough)
dfToronto=dfToronto.append(dfEtobicoke)
dfToronto=dfToronto.append(dfNorthYork)
dfToronto=dfToronto.append(dfEastYork)
dfToronto

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,51,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,53,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,54,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
...,...,...,...,...,...,...
0,35,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
1,36,M4C,East York,Woodbine Heights,43.695344,-79.318389
2,38,M4G,East York,Leaside,43.709060,-79.363452
3,39,M4H,East York,Thorncliffe Park,43.705369,-79.349372


In [45]:
toronto_venues = getNearbyVenues(names=dfToronto['Neighbourhood'],
                                   latitudes=dfToronto['Latitude'],
                                   longitudes=dfToronto['Longitude']
                                  )
toronto_venues()


Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, "Tam OShanter"
Agincourt North, "LAmoreaux East", Milliken, Steele

TypeError: 'DataFrame' object is not callable

## Now we create the Map for York(Toronto) as in previous lessons

In [46]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my_application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [47]:
# create map of Toronto using latitude and longitude values
map_of_york = folium.Map(location=[latitude, longitude], zoom_start=12)    
map_of_york


## and now we will see the hotspot(venues) per neighbourghood, crossing the map with the dfToronto venues

In [49]:
for lat, lng, borough in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc'
       ).add_to(map_of_york)

map_of_york